In [1]:
import os

In [2]:
#project working directory
%pwd

'c:\\Users\\sucha\\end-to-end-chest-classification-using-mlflow-and-dvc\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\sucha\\end-to-end-chest-classification-using-mlflow-and-dvc'

In [5]:
# This is the entity class for data ingestion
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


In [8]:
import logging
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import *

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config



    def download_file(self)-> str:
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion/", exist_ok=True)
            logger.info(f"downloading file from : {dataset_url} into : {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix + file_id, zip_download_dir)

            logger.info(f"downloaded file from : {dataset_url} into : {zip_download_dir}")

        except Exception as e:
            raise e
    

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-18 19:43:47,237: INFO: common: yaml: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://drive.google.com/file/d/1qYMa-gesEEwyRnQgK7wiEocOlznLqy58/view?usp=sharing', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}]
[2023-12-18 19:43:47,242: INFO: common: yaml: {'key': 'val'}]
[2023-12-18 19:43:47,244: INFO: common: created directory at: artifacts]
[2023-12-18 19:43:47,245: INFO: common: created directory at: artifacts/data_ingestion]
[2023-12-18 19:43:47,266: INFO: 1836773107: downloading file from : https://drive.google.com/file/d/1qYMa-gesEEwyRnQgK7wiEocOlznLqy58/view?usp=sharing into : artifacts/data_ingestion/data.zip]


Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1qYMa-gesEEwyRnQgK7wiEocOlznLqy58
From (redirected): https://drive.google.com/uc?/export=download&id=1qYMa-gesEEwyRnQgK7wiEocOlznLqy58&confirm=t&uuid=fae7a4a0-633f-4d88-a4fb-9f412095eee3
To: c:\Users\sucha\end-to-end-chest-classification-using-mlflow-and-dvc\artifacts\data_ingestion\data.zip
100%|██████████| 71.3M/71.3M [00:02<00:00, 31.1MB/s]

[2023-12-18 19:43:50,812: INFO: 1836773107: downloaded file from : https://drive.google.com/file/d/1qYMa-gesEEwyRnQgK7wiEocOlznLqy58/view?usp=sharing into : artifacts/data_ingestion/data.zip]
